In [1]:
## Chapter 10: Pattern
## You have a grid of squares, which must all be filled in either black or white. Beside each row of the grid are listed the lengths of the runs of black squares on that row; above each column are listed the lengths of the runs of black squares in that column. Your aim is to fill in the entire grid black or white.
##
## I first saw this puzzle form around 1995, under the name ‘nonograms’. I've seen it in various places since then, under different names.
##
## Normally, puzzles of this type turn out to be a meaningful picture of something once you've solved them. However, since this version generates the puzzles automatically, they will just look like random groupings of squares. (One user has suggested that this is actually a good thing, since it prevents you from guessing the colour of squares based on the picture, and forces you to use logic instead.) The advantage, though, is that you never run out of them.
##
## 10.1 Pattern controls
## This game is played with the mouse.
##
## Left-click in a square to colour it black. Right-click to colour it white. If you make a mistake, you can middle-click, or hold down Shift while clicking with any button, to colour the square in the default grey (meaning ‘undecided’) again.
##
## You can click and drag with the left or right mouse button to colour a vertical or horizontal line of squares black or white at a time (respectively). If you click and drag with the middle button, or with Shift held down, you can colour a whole rectangle of squares grey.
##
## You can also move around the grid with the cursor keys. Pressing the return key will cycle the current cell through empty, then black, then white, then empty, and the space bar does the same cycle in reverse.
##
## Moving the cursor while holding Control will colour the moved-over squares black. Holding Shift will colour the moved-over squares white, and holding both will colour them grey.
##
## (All the actions described in section 2.1 are also available.)
## 
## 10.2 Pattern parameters
##The only options available from the ‘Custom...’ option on the ‘Type’ menu are Width and Height, which are self-explanatory.

In [2]:
## Created on 1/8/2021
## Takács Tamás

In [3]:
import csp
import random
import search
import utils
import time

In [4]:
## SETTING UP CONSTRAINTS FOR 5 DIFFERENT GAMES
average = 0
constraints = []
constraints.append([[[1,3],[1],[1],[1,1],[1,1]],[[1,1],[1,1],[1,1],[1,1],[1,1]]])
constraints.append([[[5],[2],[1,1],[1,1],[1,1]],[[5],[2],[1,1],[1,1],[1,1]]])
constraints.append([[[1,1,1],[1,1],[1,1,1],[1,1],[1,1,1]],[[1,1,1],[1,1],[1,1,1],[1,1],[1,1,1]]])
constraints.append([[[3],[1],[3],[1],[3]],[[1,1],[1,1],[5],[1],[1]]])
constraints.append([[[1,1],[1,1],[1],[2,1],[1,2]],[[1,2],[1,1],[1],[1,1],[2,1]]])
frame_constraints = None

In [5]:
def frame(a):
    
    ## CHECK IF SEGMENTS IN A VECTOR ARE CORRECT ACCORDING TO THE CONSTRAINTS
    def are_segments_ok(l, constr):
        found = 0
        i = 0
        while i < len(l):
            if l[i]=='b':
                found += 1
                tmpblack = 0
                for j in range(i,len(l)):
                    if l[j]!='b':
                        break
                    tmpblack += 1
                if found <= len(constr) and constr[found-1] != tmpblack:
                    return False
                i = j
            i += 1
        if(found!= len(constr)):
            return False
        return True
    
    ## GETS BOARD INDEXES WITHOUT CONSTRAINTS AND RETURNS ITS STATES (eg. NONE, B, W) 
    def rewrite(l,a):
        return [a.get(i, None) for i in l]
        
    ## CHECKS IF A CONSTRAINT IS SATISFACTORY. IF VECTOR IS FILLED AND SEGMENTS ARE NOT OK RETURN TRUE
    def check(l,a):
        lr = rewrite(l[:-1],a)
        return not None in lr and not are_segments_ok(lr, l[len(l)-1])
    
    return any(check(l,a) for l in frame_constraints)

In [6]:
## USES MINIMUM REMAINING VALUE VARIABLE ORDERING, DEFAULT VALUE ORDER AND FORWARD CHECKING INFERENCE
def backtracking_search(csp, select_unassigned_variable=csp.mrv,
                        order_domain_values=csp.unordered_domain_values,
                        inference=csp.forward_checking):

    def backtrack(assignment):
        if len(assignment) == len(csp.variables):
            return assignment
        var = select_unassigned_variable(assignment, csp)
        for value in order_domain_values(var, assignment, csp):
            assignment2 = assignment.copy()
            assignment2[var]=value
            if 0 == csp.nconflicts(var, value, assignment) and not frame(assignment2):
                csp.assign(var, value, assignment)
                removals = csp.suppose(var, value)
                if inference(csp, var, value, assignment, removals):
                    result = backtrack(assignment)
                    if result is not None:
                        return result
                csp.restore(removals)
        csp.unassign(var, assignment)
        return None

    result = backtrack({})
    assert result is None or csp.goal_test(result)
    return result

In [7]:
def PatternCSP(fcs):
    
    ## GETS NEIGHBOURS OF A GRID ELEMENT. (eg. 1,1 has 1,1...5, and 1...5,1)
    def neighbour(i,j):
        return [i*size+k for k in range(1,size+1) if j!=k]+\
               [k*size+j for k in range(size) if i!=k]
        
    ## GETS CONSTRAINTS AND CONSTRUCTS THE CURRENT FRAME FOR THE GAME
    def frame(fcs):
        l,d = fcs
        cs = []
        for i in range(size):
            c = [i*size+k for k in range(1,size+1)]
            c.append(l[i])
            cs.append(c)
            
            c=[k*size+i+1 for k in range(size)]
            c.append(d[i])
            cs.append(c)
            
        return cs
    
    ## SETS UP VARIABLES FOR MAP COLORING CSP PROBLEM
    global frame_constraints
    global average
    size = len(fcs[1])
    vars = [i for i in range(1,size*size + 1)]
    domain = ['b','w']
    neighbours = {i*size+j:neighbour(i,j) for i in range(size) for j in range(1,size+1)}
    frame_constraints = frame(fcs)
    
    ## CREATES A MAP COLORING CSP PROBLEM
    t = csp.MapColoringCSP(domain,neighbours)
    
    ## STARTS TIMER AND BEGINS THE BACKTRACKING ALGORITHM
    start_time = time.time()
    z = backtracking_search(t)
    for i in range(size):
        for j in range(size):
            print(z.get(i*size+j+1), end=" ")
        print()
    print('Time to solve: {:.3f}'.format(time.time()-start_time))
    average += time.time()-start_time
    

for constraint in constraints:
    PatternCSP(constraint)
    print()

print('Average time to solve was: {:.3f}'.format(average/len(constraints)))

b w b b b 
w b w w w 
w w w w b 
b w b w w 
w b w b w 
Time to solve: 1.724

b b b b b 
b b w w w 
b w b w w 
b w w b w 
b w w w b 
Time to solve: 2.183

b w b w b 
w b w b w 
b w b w b 
w b w b w 
b w b w b 
Time to solve: 0.497

b b b w w 
w w b w w 
b b b w w 
w w b w w 
w w b b b 
Time to solve: 1.340

w b w w b 
b w w w b 
w w w b w 
b b w w b 
b w b b w 
Time to solve: 0.687

Average time to solve was: 1.286
